# Import settings

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from time import sleep

In [2]:
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Simple functions to get content from weather website and organize into dataframe

In [ ]:
def get_content(driver,url):
    driver.get(url)
    time.sleep(30)
    content = driver.page_source.encode('utf-8')
    driver.close()
    soup = BeautifulSoup(content, 'lxml')
    return soup

In [11]:
def organize_data(soup):
    weather = []
    for st in soup.find_all('td'):
        weather.append(st.string)
    date, high_temp, low_temp, wea, wind, air = [], [], [], [], [], []
    ###############################################################################################
    for i in range(0, len(weather), 6):
        date.append(weather[i])
        high_temp.append(weather[i+1])
        low_temp.append(weather[i+2])
        wea.append(weather[i+3])
        wind.append(weather[i+4])
        air.append(weather[i+5])
    date = [i.split(' ')[0] for i in date]
    high_temp = [i.split('°')[0] for i in high_temp]
    low_temp = [i.split('°')[0] for i in low_temp]
    wea = [i.split('~') for i in wea]
    wind = [i.split('风') for i in wind]
    air = [i.split(' ') for i in air]
    df = pd.DataFrame({'日期': date, '最高温': high_temp,
                       '最低温': low_temp, '气候': wea, '风力': wind, '空气质量': air})

    return df

In [25]:
# loop click and concat


def get_data(driver, first_button, buttons):
    df = pd.DataFrame(columns= ['日期', '最高温',
                       '最低温', '气候', '风力', '空气质量'])
    for button in buttons:
        element = driver.find_element_by_xpath(first_button)
        new_cle = driver.find_element_by_xpath(button)
        element.click()
        new_cle.click()
        content = driver.page_source.encode('utf-8')
        soup = BeautifulSoup(content, 'lxml')
        new_df = organize_data(soup)
        df = pd.concat([df, new_df])
    df.set_index('日期', inplace=True)
    driver.close()
    return df

# Run functions and write the results into excel

In [24]:
driver = webdriver.Chrome()
driver.get("http://tianqi.2345.com/wea_history/59287.htm")

In [23]:
# button for June-August
first_button = '/html/body/div[8]/div[2]/div[1]/div[1]/div[1]/div[3]/div[2]'
buttons = ['/html/body/div[8]/div[2]/div[1]/div[1]/div[1]/div[3]/div[2]/div/ul/li[6]',
           '/html/body/div[8]/div[2]/div[1]/div[1]/div[1]/div[3]/div[2]/div/ul/li[7]', 
           '/html/body/div[8]/div[2]/div[1]/div[1]/div[1]/div[3]/div[2]/div/ul/li[8]',
           '/html/body/div[8]/div[2]/div[1]/div[1]/div[1]/div[3]/div[2]/div/ul/li[9]']

In [26]:
weather = get_data(driver, first_button, buttons)

,最高温,最低温,气候,风力,空气质量
日期,,,,,
2020-09-01,35,27,[多云],"[东北, 2级]","[53, 良]"
2020-09-02,35,27,[多云],"[北, 2级]","[75, 良]"
2020-09-03,36,27,[多云],"[东北, 2级]","[81, 良]"
2020-06-01,31,25,"[中雨, 雷阵雨]","[南, 2级]","[35, 优]"
2020-06-02,28,25,"[小雨, 雷阵雨]","[南, 3级]","[28, 优]"
...,...,...,...,...,...
2020-08-27,33,25,"[阴, 多云]","[北, 2级]","[42, 优]"
2020-08-28,35,27,"[多云, 晴]","[东北, 2级]","[71, 良]"
2020-08-29,34,27,[晴],"[西北, 2级]","[76, 良]"


In [48]:
with pd.ExcelWriter('天气情况.xlsx') as writer:
    weather.to_excel(writer)